In [9]:
import numpy as np
import rasterio
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import os
from netCDF4 import Dataset as NetCDFFile 
import h5netcdf
from mpl_toolkits.axes_grid1 import make_axes_locatable
import rioxarray as rio
import geopandas as gpd
import seaborn as sns
from mpl_toolkits.basemap import Basemap

In [15]:
#load in temp data
# temp_1999 = xr.open_dataset("../Temperature/Output Deliverable/export_1999.nc")
# temp_2008 = xr.open_dataset("../Temperature/Output Deliverable/export_2008.nc")
# temp_change = xr.open_dataset("../Temperature/Output Deliverable/export_change.nc")
#load in netcdf file
change = NetCDFFile('../Temperature/Output Deliverable/export_change.nc')
temp_2008 = NetCDFFile('../Temperature/Output Deliverable/export_2008.nc')
temp_1999 = NetCDFFile('../Temperature/Output Deliverable/export_1999.nc')

#load in whale data
loc_1999 = pd.read_csv('exports/loc_1999.csv').drop('geometry', axis=1)
loc_2008 = pd.read_csv('exports/loc_2008.csv').drop('geometry', axis=1)
#turn to gpd
loc_1999 = gpd.GeoDataFrame(loc_1999, geometry=gpd.points_from_xy(x=loc_1999['long'], y=loc_1999['lat']))
loc_2008 = gpd.GeoDataFrame(loc_2008, geometry=gpd.points_from_xy(x=loc_2008['long'], y=loc_2008['lat']))

In [16]:
temp_2008

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    Metadata_Conventions: CF-1.6, Unidata Dataset Discovery v1.0
    metadata_link: C00884
    id: ersst.v4.200801
    naming_authority: gov.noaa.ncdc
    title: NOAA ERSSTv4 (in situ only)
    summary: ERSST.v4 is developped based on v3b after revisions of 11 parameters using updated data sets and advanced knowledge of ERSST analysis
    institution: NOAA/NESDIS/NCEI/CCOG
    creator_name: Boyin Huang
    creator_email: boyin.huang@noaa.gov
    date_modified: 2020/03/03
    production_version: Version 5
    history: Tue Mar  3 13:41:26 2020: ncatted -O -a _FillValue,ssta,o,f,-999.0 ssta.nc
Version 5 based on Version 4
    publisher_name: Boyin Huang
    publisher_email: boyin.huang@noaa.gov
    creator_url: http://www.ncdc.noaa.gov
    license: No constraints on data access or use
    time_coverage_start: 2008-01-15T000000Z
    time_coverage_end: 2008-01-15T000000Z
    geospat

In [28]:
#read the variables in the netcdf files:

#change map
lat = change.variables['lat'][:] 
long = change.variables['lon'][:]
sst = change.variables['change_percent'][:] 

#

lat_1999 = temp_1999.variables['lat'][:] 
long_1999 = temp_1999.variables['lon'][:]
sst_1999 = temp_1999.variables['sst'][:][0][0][0]

lat_2008= temp_2008.variables['lat'][:]
long_2008 = temp_2008.variables['lon'][:]
sst_2008 = temp_2008.variables['sst'][:] [0][0][0]

# Define the corners of the map
llon = -165.662
llat = 4.606
ulon = -54.362
ulat = 53.148

In [29]:
sst_2008

masked_array(data=[27.39490509033203, 27.292213439941406,
                   27.194772720336914, 27.10675811767578,
                   27.032577514648438, 26.968748092651367,
                   26.90727424621582, 26.84893035888672,
                   26.803319931030273, 26.770490646362305,
                   26.74452018737793, 26.727689743041992,
                   26.695039749145508, 26.66314697265625,
                   26.64833641052246, 26.6466064453125,
                   26.635019302368164, 26.60792350769043,
                   26.581689834594727, 26.566022872924805,
                   26.551462173461914, 26.53693199157715,
                   26.545068740844727, 26.6156005859375,
                   26.69529914855957, 26.762420654296875,
                   26.815526962280273, 26.89735221862793,
                   26.930078506469727, 26.949602127075195,
                   26.983253479003906, 27.04386329650879,
                   27.107690811157227, 27.173118591308594,
             

In [ ]:


# Create a Basemap object
map = Basemap(projection='merc', llcrnrlon=llon, llcrnrlat=llat, urcrnrlon=ulon, urcrnrlat=ulat, resolution='f')

# Draw map features
map.drawcoastlines()
map.drawcountries()
map.drawlsmask(land_color='Linen', ocean_color='#CCFFFF')

# Generate some sample data (replace this with your actual data)
lons = long - 360
lats = lat
lons, lats = np.meshgrid(lons, lats)
sst = sst  # Sample sea surface temperature data

# Transform coordinates for plotting on the map
x, y = map(lons, lats)

# Plot the contour plot on the map
temp = map.contourf(x, y, sst)

# Generate some sample data points to overlay on the map
data_lons = loc_2008.long
data_lats = loc_2008.lat

# Transform coordinates for plotting data points on the map
data_x, data_y = map(data_lons, data_lats)

# Plot the data points on the map
map.plot(data_x, data_y, 'or', markersize=1)  # blue circles with size 5

# Add a colorbar
cb = map.colorbar(temp, location='bottom', size="5%", pad="2%")
cb.set_label('Percent Change')

plt.clabel(temp, fontsize=9, inline=1) # contour labels
plt.title('Percent change in Temperature: 1999 - 2008')
# Show the plot
plt.show()
plt.savefig('exports/change_2008_with_whales.png');

In [ ]:
# Create a Basemap object
map = Basemap(projection='merc', llcrnrlon=llon, llcrnrlat=llat, urcrnrlon=ulon, urcrnrlat=ulat, resolution='f')

# Draw map features
map.drawcoastlines()
map.drawcountries()
map.drawlsmask(land_color='Linen', ocean_color='#CCFFFF')

# Transform coordinates for plotting on the map
x, y = map(lons, lats)

# Plot the contour plot on the map
temp = map.contourf(x, y, sst)

# Add 1999 data 
data_lons = loc_2008.long
data_lats = loc_2008.lat
data_lons_1 = loc_1999.long
data_lats_1 = loc_1999.lat
# Transform coordinates for plotting data points on the map
data_x, data_y = map(data_lons, data_lats)
data_x_1, data_y_1 = map(data_lons_1, data_lats_1)
# Plot the data points on the map
map.plot(data_x, data_y, 'or', markersize=1)  # blue circles with size 5
map.plot(data_x_1, data_y_1, 'ob', markersize=1)

# Add a colorbar
cb = map.colorbar(temp, location='bottom', size="5%", pad="2%")
cb.set_label('Percent Change')

plt.clabel(temp, fontsize=9, inline=1) # contour labels
plt.title('Temp. Change and Whale Locations: 1999 - 2008 ')
# Show the plot
plt.show()
plt.savefig('exports/change_1999_2008_with_whales.png');